In [ ]:
import numpy as np
import torch
from sklearn.linear_model import LinearRegression

A tensor is a number, vector, matrix or any n-dimensional array.

## Problem Statement

We'll create a model that predicts crop yeilds for apples (*target variable*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in different regions. 

Here's the training data:

>Temp | Rain | Humidity | Prediction
>--- | --- | --- | ---
> 73 | 67 | 43 | 56
> 91 | 88 | 64 | 81
> 87 | 134 | 58 | 119
> 102 | 43 | 37 | 22
> 69 | 96 | 70 | 103

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
```

It means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.



**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

## Training Data
The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column for variable.

In [ ]:
# Input (temp, rainfall, humidity)
ip = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [ ]:
# Target (apples)
target = np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.

In [ ]:
#converting input and target to tensors
ip_tensor = torch.tensor(ip)
print('Tensor Inputs  :  \n', ip_tensor)
target_tensor = torch.tensor(target)
print('\nTensor targets  :  \n', target_tensor)

Tensor Inputs  :  
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

Tensor targets  :  
 tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


## Linear Regression Model (from scratch)

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [ ]:
# Weights and biases
weights = torch.randn(1, 3, requires_grad=True)
biases = torch.randn(1, requires_grad=True)
weights_transpose = torch.transpose(weights, 0, 1)

print("Weights  :  ",weights)
print("\nBiases  :  ",biases)
print("\nWeights_transpose  :  \n",weights_transpose)

Weights  :   tensor([[ 0.7125,  0.0496, -0.3919]], requires_grad=True)

Biases  :   tensor([-0.5541], requires_grad=True)

Weights_transpose  :  
 tensor([[ 0.7125],
        [ 0.0496],
        [-0.3919]], grad_fn=<TransposeBackward0>)


The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `w0` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T 
$$

$$
\left[ \begin{array}{cc}
1 & 73 & 67 & 43 \\
1 &91 & 88 & 64 \\
\vdots & \vdots & \vdots & \vdots \\
1 &69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{0}  \\
w_{1}  \\
w_{2}  \\
w_{3} 
\end{array} \right]
%
$$

In [ ]:
def fit(x, w_t, b):
  return (x @ w_t) + b

The matrix obtained by passing the input data to the model is a set of predictions for the target variables.

In [ ]:
#predictions
pred = fit(ip_tensor, weights_transpose, biases)
print(pred)

tensor([[37.9306],
        [43.5673],
        [45.3479],
        [59.7566],
        [25.9363]], grad_fn=<AddBackward0>)


Compare with targets

In [ ]:
print(target_tensor)

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


Because we've started with random weights and biases, the model does not perform a good job of predicting the target varaibles.

## Loss Function

We can compare the predictions with the actual targets, using the following method: 
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [ ]:
def mse(pred, target):
  difference = pred - target
  return torch.sum(difference * difference) / difference.numel()

In [ ]:
def model(x, w):
    return x @ w.T

In [ ]:
#compute loss
loss = mse(pred, target_tensor)
print('Loss   :  ', loss)

Loss   :   tensor(2903.3398, grad_fn=<DivBackward0>)


The resulting number is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have requires_grad set to True.

More on autograd: https://pytorch.org/docs/stable/autograd.html#module-torch.autograd



In [ ]:
#compute gradient
loss.backward()

The gradients are stored in the .grad property of the respective tensors.

In [ ]:
# Gradients for weights
print(weights, end="\n-------------\n")
print(weights.grad)

tensor([[ 0.7125,  0.0496, -0.3919]], requires_grad=True)
-------------
tensor([[-5039.7568, -8059.4722, -4576.7837]])


A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases. 

* If a gradient element is **postive**, 
    * **increasing** the element's value slightly will **increase** the loss.
    * **decreasing** the element's value slightly will **decrease** the loss.




* If a gradient element is **negative**,
    * **increasing** the element's value slightly will **decrease** the loss.
    * **decreasing** the element's value slightly will **increase** the loss.
    


The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [ ]:
weights.grad.zero_()
biases.grad.zero_()
print(weights.grad, end="\n-------\n")
print(biases.grad)

tensor([[0., 0., 0.]])
-------
tensor([0.])


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [ ]:
#prediction
prediction = fit(ip_tensor, weights.t(), biases)
print(prediction)

tensor([[37.9306],
        [43.5673],
        [45.3479],
        [59.7566],
        [25.9363]], grad_fn=<AddBackward0>)


In [ ]:
#loss
loss = mse(prediction, target_tensor)
print(loss)

tensor(2903.3398, grad_fn=<DivBackward0>)


In [ ]:
#compute gradients
loss.backward() 

In [ ]:
# Adjust weights & reset gradients
with torch.no_grad():
    weights -= weights.grad * 1e-5
    biases -= biases.grad * 1e-5
    weights.grad.zero_()
    biases.grad.zero_()
print(weights)

tensor([[ 0.7629,  0.1301, -0.3461]], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [ ]:
# Calculate loss
prediction = fit(ip_tensor, weights.t(), biases)
loss = mse(prediction, target_tensor)
print(loss)

tensor(1983.8226, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [ ]:
# Train for 100 epochs
for i in range(100):
    prediction = fit(ip_tensor, weights.t(), biases)
    loss = mse(prediction, target_tensor)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero_()
        biases.grad.zero_()

In [ ]:
# Calculate loss
prediction = fit(ip_tensor, weights.t(), biases)
loss = mse(prediction, target_tensor)
print(loss)

tensor(157.3732, grad_fn=<DivBackward0>)


In [ ]:
#prediction
print(prediction)

tensor([[ 60.8336],
        [ 79.3799],
        [119.2232],
        [ 42.4101],
        [ 84.4459]], grad_fn=<AddBackward0>)


In [ ]:
#tensor targets
print(target_tensor)

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])
